In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
import os

#####  1. 导入数据

In [7]:
save_dir = '/home/aily/traindata/mnist'
if os.path.exists(save_dir) is False:
    os.mkdirs(save_dir)
mnist = input_data.read_data_sets(save_dir, one_hot=True)

Extracting /home/aily/traindata/mnist/train-images-idx3-ubyte.gz
Extracting /home/aily/traindata/mnist/train-labels-idx1-ubyte.gz
Extracting /home/aily/traindata/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/aily/traindata/mnist/t10k-labels-idx1-ubyte.gz


##### 2. 设置变量

In [8]:
# 创建x, x是一个占位符(placeHolder),代表待识别的图片,等使用的时候才加载，分配空间。
x = tf.placeholder(tf.float32, [None, 784])
# y_是实际的图像标签，同样以占位符表示
y_ = tf.placeholder(tf.float32, [None, 10])

#由于使用cnn，将784转化为28*28矩阵,其中-1是数量，根据x的数量决定的。x_image就是训练图像
x_image = tf.reshape(x, [-1, 28, 28, 1])

##### 3. 定义卷积

In [9]:
#初始化权重参数,初始值为正态分布
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

#初始化偏置参数，初始值全为0.1
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

#定义卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME')

#定义池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding = 'SAME')

##### 5. 使用卷积
1. weight_variable([5,5,1,32])
 - 1. 前面两个5,表示卷积核的长宽分别为5
 - 2. 1表示输入图像对应的通道数，若输入的图像是单通道的则设置为1，如果是RGB三通道的，则设置为3
 - 3. 32表示卷积核的个数，对应输出32张图
2. strides=[1, 1, 1, 1]
 - 四个元素规定前后必须为1，中间两个数表示水平滑动和垂直滑动步长值
3. padding='SAME'
 - SAME表示在扫描的时候，如果遇到卷积核比剩下的元素要大时，这个时候需要补0进行最后一次的行扫描或者列扫描
4.  ksize=[1,2,2,1]
 - 池化核大小

In [10]:
#第一层卷积
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#第二层卷积
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

#全连接层,输出1024维度的向量
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
#使用dropout，keep_prob是个占位符，训练时时0.5,测试时1
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

#全连接层，转换为类别打分。1024维度向量转化为10维度，对应10类
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

W0818 21:29:32.630597 140478835910464 deprecation.py:506] From <ipython-input-10-a33c12fca8d2>:20: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


##### 6. 计算损失

In [11]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv)
                              )
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#定义测试准确率
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

W0818 21:29:32.656077 140478835910464 deprecation.py:323] From <ipython-input-11-e0da1f45bb07>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



##### 7. 计算
eval() 其实就是tf.Tensor的Session.run() 的另外一种写法

In [12]:
#初始化变量
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

#训练
for i in range(20000):
    batch = mnist.train.next_batch(100)
    #每100次计算一次准确率
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], keep_prob:1.0})
        print("step %d, training accuracy %g" % (i, train_accuracy))
    train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob: 0.5})
    
print("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))

step 0, training accuracy 0.05
step 100, training accuracy 0.89
step 200, training accuracy 0.91
step 300, training accuracy 0.94
step 400, training accuracy 0.92
step 500, training accuracy 0.95
step 600, training accuracy 0.97
step 700, training accuracy 0.98
step 800, training accuracy 0.98
step 900, training accuracy 0.94
step 1000, training accuracy 0.99
step 1100, training accuracy 0.98
step 1200, training accuracy 0.97
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.97
step 1600, training accuracy 0.95
step 1700, training accuracy 0.97
step 1800, training accuracy 0.96
step 1900, training accuracy 0.97
step 2000, training accuracy 0.98
step 2100, training accuracy 0.99
step 2200, training accuracy 1
step 2300, training accuracy 0.99
step 2400, training accuracy 0.95
step 2500, training accuracy 1
step 2600, training accuracy 1
step 2700, training accuracy 0.99
step 2800, training accuracy 0.97
step 2900, training accuracy 0.97
s